### Importing Libraries

In [8]:
!pip install llama-index

In [9]:
!pip install pypdf

In [10]:
!pip install opencv-python
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils
!apt update
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip3 install opencv-contrib-python==4.5.5.62
!pip install --upgrade pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building

In [11]:
!pip install transformers

In [12]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

### Text Extraction from Image

In [13]:
import cv2
import pytesseract
from pdf2image import convert_from_path
from google.colab.patches import cv2_imshow
import os
import numpy as np

In [14]:
def extract_from_image(path):
  import os
  image = cv2.imread(path)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)
  # Adaptive thresholding
  thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)
  # Adjust contrast
  contrast = 0.95  # Adjust the value as desired (between 0 and 1)
  adjusted_image = np.clip(thresh.astype(np.float32) * contrast, 0, 255).astype(np.uint8)
  text = str(pytesseract.image_to_string(adjusted_image, config='--psm 6'))
  # Dilation
  #kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  #dilated = cv2.dilate(thresh, kernel, iterations=1)
  # Display the enhanced image
  #cv2_imshow(adjusted_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  image_p
  os.mkdir()
  with open("image_text.txt","w+") as f:
    f.write(text)
  return os.path.abspath("image_text.txt")


### Making Llama2 Query Engine

In [15]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [16]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16408651542469187887
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14626652160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14264690936307915475
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [17]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

In [18]:
model_url = 'https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf'

In [19]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=1500,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4783.16


4562it [00:25, 182.36it/s]                          
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '17'}


In [20]:
# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [21]:
# create a service context
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

In [1]:
#function to save file in colab
def save_file():
  # Directory
  directory = "document"

  # Parent Directory path
  parent_dir = "/content/documents"

  # Path
  path = os.path.join(parent_dir, directory)
  os.mkdir(path)

In [46]:
#function which checks the extension of file
def check_ext(path):
  f = os.listdir(path)[0]x
  if f.endswith('.pdf'):
    document = SimpleDirectoryReader(path).load_data()
  elif f.endswith('.jpg') or path.endswith('png'):
    txt_path = extract_from_image(f)
    document = SimpleDirectoryReader(txt_path).load_data()
  else:
    return None
  return document


In [48]:
print(check_ext('/content/document'))

ValueError: Directory RAG-Document.pdf does not exist.

In [43]:
#function to pass query to llama2
def get_response(document, ques):
  index = VectorStoreIndex.from_documents(document, service_context=service_context)
  query_engine = index.as_query_engine()
  response = query_engine.query(ques)
  return response

In [44]:
def main():
  path = input("Enter the Document/Image you wish to query ")
  doc = check_ext(path)
  ques = input("Enter the Document/Image you wish to query ")
  response = get_response(doc,ques)
  print(response)

In [45]:
main()

Enter the Document/Image you wish to query /content/document


KeyboardInterrupt: Interrupted by user